In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from math import *


In [12]:
import itertools
def parse_file(file_path, startLine, endLine):
    """
    Parse the file starting at startLine and ending at endLine.
    """
    # Create an empty list to store the parsed data
    parsed_data = []

    # Open the file for reading
    with open(file_path, 'r') as file:
        # Read the file line by line starting at line startLine
        for line in itertools.islice(file, startLine, endLine):
            # Strip newline characters and split the line into columns
            columns = line.strip().split('|||')
            # Add the parsed columns to the parsed_data list
            parsed_data.append(columns)
    
    return parsed_data

def get_ppdb2_score(array):
    features = array[3].split(' ')
    for feature in features:
        if feature.startswith('PPDB2'):
            return feature.split('=')[1]


def get_ppdb1_score(array):
    features = array[3].split(' ')
    for feature in features:
        if feature.startswith('PPDB1'):
            return feature.split('=')[1]
            


In [1]:
file = 'ppdb-2.0-xxxl-all'
lineCount = 0
with open(file, 'r') as f:
    for line in f:
        lineCount += 1
print(lineCount)


40138455


In [14]:
file = 'ppdb-2.0-xxxl-all'
groupSize = 100000
# data = parse_file(file)

# Create a DataFrame from the parsed data
# print(data[0])
# df = dd.from_pandas(pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'features', 'alignment', 'entailment']), chunksize=10000)
# df = pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'features', 'alignment', 'entailment'])
# df

for i in range(0, floor(lineCount/groupSize)):
    data = parse_file(file, i*groupSize, (i+1)*groupSize)
    p1scores = []
    for j in range(len(data)):
        p1scores.append(get_ppdb1_score(data[j]))
    p2scores = []
    for j in range(len(data)):
        p2scores.append(get_ppdb2_score(data[j]))
    df = pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'temp', 'alignment', 'entailment']).drop(columns=['temp'])
    df['ppdb1.0 score'] = p1scores
    df['ppdb2.0 score'] = p2scores
    df.to_csv('ppdb-xxxl-all-'+str(i)+'.csv')
if lineCount%groupSize != 0:
    data = parse_file(file, floor(lineCount/groupSize)*groupSize, lineCount)
    p1scores = []
    for j in range(len(data)):
        p1scores.append(get_ppdb1_score(data[j]))
    p2scores = []
    for j in range(len(data)):
        p2scores.append(get_ppdb2_score(data[j]))
    df = pd.DataFrame(data, columns=['lhs', 'phrase', 'paraphrase', 'temp', 'alignment', 'entailment']).drop(columns=['temp'])
    df['ppdb1.0 score'] = p1scores
    df['ppdb2.0 score'] = p2scores
    df.to_csv('ppdb-xxxl-all-'+str(floor(lineCount/groupSize)+1)+'.csv')



In [15]:
df = dd.read_csv('ppdb-xxxl-all-*.csv')
df

,Unnamed: 0,lhs,phrase,paraphrase,alignment,entailment,ppdb1.0 score,ppdb2.0 score
npartitions=402,,,,,,,,
,int64,object,object,object,object,object,float64,float64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [16]:
df = df.drop(['Unnamed: 0'], axis=1)
df


,lhs,phrase,paraphrase,alignment,entailment,ppdb1.0 score,ppdb2.0 score
npartitions=402,,,,,,,
,object,object,object,object,object,float64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [17]:
df.partitions[0].compute()

,lhs,phrase,paraphrase,alignment,entailment,ppdb1.0 score,ppdb2.0 score
0,[NN],transplant,transplantation,0-0,OtherRelated,3.29590,5.24981
1,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,0-0,Equivalence,9.46380,5.22098
2,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,0-0,Equivalence,9.46382,5.21828
3,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,0-0,Equivalence,9.46378,5.21462
4,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1...,0-0,Equivalence,9.46380,5.21153
...,...,...,...,...,...,...,...
99995,[VBN],secretary,addressed,0-0,Independent,22.02127,3.60010
99996,[VBZ],guarantees,enshrines,0-0,Independent,12.66003,3.60010
99997,[VBG],collecting,extracting,0-0,Independent,24.53596,3.60009
99998,[VBZ],governs,observes,0-0,Independent,22.79739,3.60009


In [18]:
df.to_csv('ppdb-xxxl-all.csv')

['/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/000.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/001.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/002.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/003.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/004.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/005.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/006.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/007.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/008.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/009.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/010.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/011.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/012.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/013.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/014.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/015.part',
 '/home/andrew/Documents/ppdb/ppdb-xxxl-all.csv/016.part